In [1]:
"""
elasticsearchstore使用教程： https://api.python.langchain.com/en/latest/vectorstores/langchain_elasticsearch.vectorstores.ElasticsearchStore.html
"""
import os, json
import numpy as np

from datasets import load_dataset
from FlagEmbedding import FlagModel


/root/autodl-tmp/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

all_id_list = []
all_query_list = []
# all_response_list = []
# all_metadata_list = []
# all_score_list = []

all_metadata_list = []

with open("FreedomIntelligence___huatuo26_m-lite/format_data.jsonl", "r", encoding="utf-8") as fin:
    for line in fin:
        # print(line.strip())
        example_qa = json.loads(line)
        
        all_id_list.append(example_qa["id"])
        all_query_list.append(example_qa["question"])

        all_metadata_list.append({"answer": example_qa["answer"], "query": example_qa["question"], "score": example_qa["score"], "related_diseases": example_qa["related_diseases"]})

        
        # all_response_list.append(example_qa["response"])
        # all_score_list.append(example_qa["score"])
        # all_metadata_list.append()
        

print(len(all_query_list), len(all_metadata_list))

177703 177703


In [3]:


from langchain_community.vectorstores import ElasticsearchStore


from langchain.embeddings import XinferenceEmbeddings
# from langchain.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings, HuggingFaceInferenceAPIEmbeddings

# 路由中不要包含v1
xinference_embeddings = XinferenceEmbeddings(server_url="http://localhost:9997",model_uid="bge-base-zh-v1.5")

es_store = ElasticsearchStore(index_name="yiliao_index", es_url="http://localhost:9200", embedding=xinference_embeddings)

print(es_store)



/tmp/ipykernel_9856/2487139774.py:10: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use Use class in langchain-elasticsearch package instead.
  es_store = ElasticsearchStore(index_name="yiliao_index", es_url="http://localhost:9200", embedding=xinference_embeddings)


In [4]:
# print(es_store.delete())

es_store.similarity_search(query="感冒")

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [yiliao_index]', yiliao_index, index_or_alias)

In [5]:

# 读取json ,并放入到es 中

from modelscope import snapshot_download
embedding_model_dir = snapshot_download("AI-ModelScope/bge-base-zh-v1.5", revision='master', cache_dir="./bge-base-zh-v1.5")


# embedding_model_dir  = "./bge-base-zh-v1.5"

embedding_model = FlagModel(embedding_model_dir, query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True)


print("转换embedding")
query_embeddings = embedding_model.encode_corpus(all_query_list, batch_size=512)





2024-10-14 22:34:43,378 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


转换embedding


Inference Embeddings: 100%|██████████| 348/348 [01:48<00:00,  3.21it/s]


In [6]:
print(len(query_embeddings[0]))

768


In [7]:
print("插入到es中")

all_embedding_text_list = [[all_query_list[i], query_embeddings[i]] for i in range(len(all_query_list))]

doc_id_result = es_store.add_embeddings(text_embeddings=all_embedding_text_list,metadatas=all_metadata_list, ids=all_id_list)



插入到es中
